In [23]:
import rasterio
from rasterio.features import shapes
from rasterio.plot import show
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import open3d as o3d
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

In [24]:
# gdal_calc.py -A DSM/GeoTIFF/DHMVII_DSM_1m_testdata.tif -B ./dtm_1m.tif --outfile=CHM.tif --calc="A-B" --NoDataValue=0
# gdalwarp  DTM/GeoTIFF/DHMVII_DTM_5m_testdata.tif test.tif -tr 1 1 -r near
# gdal_contour -3d -a elev chm.tif contour.shp -i 0.5 -snodata 0b

In [25]:
from pyproj import Transformer
def lambert_to_wgs(x_lambert, y_lambert):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326")
    x_wgs, y_wgs = transformer.transform(x_lambert, y_lambert)
    return x_wgs, y_wgs

import requests
def search_address(cp, rue, num, as_wgs=False, as_dict=False):    
    url = 'http://geoservices.wallonie.be/geolocalisation/rest/getPositionByCpRueAndNumero/{cp}/{rue}/{num}'
    r = requests.get(url.format(cp=cp, rue=rue, num=num)).json()
    x, y, xMin, xMax, yMin, yMax = r['x'], r['y'], r['rue']['xMin'], r['rue']['xMax'], r['rue']['yMin'], r['rue']['yMax']
    if as_wgs:
        x, y       = lambert_to_wgs(x, y)
        xMin, xMax = lambert_to_wgs(xMin, xMax)
        yMin, yMax = lambert_to_wgs(yMin, yMax)
    if as_dict:
        return {'x': x, 'xMin': xMin, 'xMax': xMax, 
                'y': y, 'yMin': yMin, 'yMax': yMax}
    return x, y, xMin, xMax, yMin, yMax

In [50]:
x, y, xMin, xMax, yMin, yMax = search_address(cp='5000', rue='RUE DE LA CRETE', num = '111', as_wgs=False)

In [74]:
xMin, xMax, yMin, yMax

(185034.777599998, 187088.6096, 129235.137499999, 131179.935800001)

In [67]:
xMin, xMax, yMin, yMax = xMin-250, xMax+1000, yMin-250, yMax+1000

In [68]:
# Christophe Schenke 
# christophe.shanke@spw.walonnie.be  
# 0475 95 4570

In [69]:
#from fiona.transform import transform_geom
#hits[0][1]['geometry']  = transform_geom("epsg:31370", "epsg:4326", hits[0][1]['geometry'])

In [70]:
import fiona
gdb = fiona.open('./BATI3D_2013-2014_FILEGDB_31370/BATI3D_2013-2014.gdb', layer=1)
hits = list(gdb.items(bbox=(xMin, yMin, xMax, yMax)))

In [71]:
%%time
import open3d as o3d
meshes = []
for hit in hits:
    points = hit[1]['geometry']['coordinates']
    pts = [item for sublist in points for sitem in sublist for item in sitem]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    mesh = pcd.compute_convex_hull()
    meshes.append(mesh[0])
o3d.visualization.draw_geometries(meshes, mesh_show_wireframe=True, mesh_show_back_face=True)

CPU times: user 59 s, sys: 6.97 s, total: 1min 5s
Wall time: 1min 31s


In [12]:
def geocode(address):
    mapbox_api_key = 'pk.eyJ1Ijoia3Jva3JvYiIsImEiOiJjam83MjVrbWkwbWNoM3FwN2VhMm81eGRzIn0.yM3wkq5LJd8NeSYyPyTY4w'
    g = geocoder.mapbox(address, key=mapbox_api_key)
    return (g.json['lat'], g.json['lng'])

[[[(185185.49300000072, 129229.93600000069, 90.10000000000582),
   (185181.67700000107, 129235.54800000042, 90.10000000000582),
   (185173.68400000036, 129230.37999999896, 90.10000000000582),
   (185177.53299999982, 129224.68699999899, 90.10000000000582),
   (185184.7580000013, 129229.48600000143, 90.10000000000582),
   (185185.49300000072, 129229.93600000069, 90.10000000000582)]],
 [[(185184.7580000013, 129229.48600000143, 90.10000000000582),
   (185184.7580000013, 129229.48600000143, 100.62810000000172),
   (185185.49300000072, 129229.93600000069, 100.62810000000172),
   (185185.49300000072, 129229.93600000069, 90.10000000000582),
   (185184.7580000013, 129229.48600000143, 90.10000000000582)]],
 [[(185177.53299999982, 129224.68699999899, 90.10000000000582),
   (185177.53299999982, 129224.68699999899, 100.62810000000172),
   (185184.7580000013, 129229.48600000143, 100.62810000000172),
   (185184.7580000013, 129229.48600000143, 90.10000000000582),
   (185177.53299999982, 129224.6869999

In [73]:
[item for sublist in hits[0][1]['geometry']['coordinates'] for sitem in sublist for item in sitem]

[(185185.49300000072, 129229.93600000069, 90.10000000000582),
 (185181.67700000107, 129235.54800000042, 90.10000000000582),
 (185173.68400000036, 129230.37999999896, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185185.49300000072, 129229.93600000069, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 100.62810000000172),
 (185185.49300000072, 129229.93600000069, 100.62810000000172),
 (185185.49300000072, 129229.93600000069, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 100.62810000000172),
 (185184.7580000013, 129229.48600000143, 100.62810000000172),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 90.10000000000582),
 (185173.6